This is the jupiter notebook that Zhou writes some test functions

In [38]:
# try to define the function gradient and hessian 

import numpy as np
import scipy

from optimize_obj import *


####################################################################
# the following is the quad function
def quad(x, Q, q):
    # dim is the dimension of the problem
    # x is numpy array
    return 0.5 * np.matmul(x, np.matmul(Q, x.transpose())) + np.dot(q, x)

def quad_grad(x, Q, q):
    return np.matmul(Q, x) + q

def quad_hessian(x, Q, q):
    return Q





def quad_obj(Q, q):
    dim = len(q)
    quad_val = lambda x: quad(x, Q, q)
    quad_grad_val = lambda x: quad_grad(x, Q, q)
    quad_hessian_val = lambda x: quad_hessian(x, Q, q)
    return Obj(dim, quad_val, quad_grad_val, quad_hessian_val)

q = quad_obj(np.array([[1, 0], [0, 1]]), np.array([1, 1]))


In [40]:
x = np.linspace(-10, 10, 100)
np.matmul(x, x)

3400.6734006734005

In [69]:
def special_quad(x, Q, sigma):
    val = 0.5 * np.matmul(x, x) + 0.25 * sigma * np.power(np.matmul(x, np.matmul(Q, x)), 2)
    return val

def special_quad_grad(x, Q, sigma):
    return x + 0.5 * sigma * np.matmul(Q, x) * np.matmul(x, np.matmul(Q, x))

def special_quad_hessian(x, Q, sigma):
    return np.eye(len(x)) + sigma * np.power(np.matmul(x, np.matmul(Q, x)), 2) + 0.5 * sigma * np.matmul(x, np.matmul(Q, x)) * Q

def special_quad_obj(Q, sigma):
    dim = len(Q)
    print(dim)
    quad_val = lambda x: special_quad(x, Q, sigma)
    quad_grad_val = lambda x: special_quad_grad(x, Q, sigma)
    quad_hessian_val = lambda x: special_quad_hessian(x, Q, sigma)
    return Obj(dim, quad_val, quad_grad_val, quad_hessian_val)


spq = special_quad_obj(np.array([[1, 0, 1], [0, 1, 2], [0, 0, 12]]), 1)
spq.hessian(np.array([1, 1, 1]))    

3


array([[298.5, 289. , 297.5],
       [289. , 298.5, 306. ],
       [289. , 289. , 392. ]])

In [63]:
def data_fit(x, y):
    return np.sum((y - x[0] * (1 - x[1]))**2)

def data_fit_grad(x, y):
    # parital derivative with respect to x[0], x[1]
    return np.array([2 * np.sum((y - x[0] * (1 - x[1])) * (x[1] - 1)), 2 * np.sum((y - x[0] * (1 - x[1])) * x[0])])

def data_fit_hessian(x, y):
    f_xx = 2 * np.sum((1 - x[1])**2)
    f_yy = 2 * np.sum(x[0]**2)
    f_xy = 2 * np.sum(y - 2 * x[0] + 2 * x[0] * x[1])
    return np.array([[f_xx, f_xy], [f_xy, f_yy]])

                    
def data_fit_obj(y):
    dim = 2
    data_fit_val = lambda x: data_fit(x, y)
    data_fit_grad_val = lambda x: data_fit_grad(x, y)
    data_fit_hessian_val = lambda x: data_fit_hessian(x, y)
    return Obj(dim, data_fit_val, data_fit_grad_val, data_fit_hessian_val)

data_fit_obj(np.array([1, 1])).hessian(np.array([1, 1]))
    


array([[0, 4],
       [4, 2]])

In [73]:
def exponential(dim, x):
    # exponential function is defined as
    # frac{exp(x_1) - 1}{exp(x_1) + 1} + 0.1 exp{-x_1} + sum_{i=2}^d (x_i - 1)^4
    val = (np.exp(x[0]) - 1) / (np.exp(x[0]) + 1) + 0.1 * np.exp(-x[0]) + np.sum((x[1:] - 1)**4)
    return val

def exponential_grad(dim, x):
    # gradient of the exponential function
    grad = np.zeros(dim)
    grad[0] = (2 * np.exp(x[0])) / (np.exp(x[0]) + 1)**2 - 0.1 * np.exp(-x[0]) 
    grad[1:] = 4 * (x[1:] - 1)**3
    return grad

def exponential_hessian(dim, x):
    # hessian of the exponential function
    hessian = np.zeros((dim, dim))
    hessian[0, 0] = (2 * np.exp(x[0]) * np.power(np.exp(x[0]) + 1, 2) - 4 * np.exp(x[0]) * (np.exp(x[0] + 1))) / np.power(np.exp(x[0]) + 1, 4) + 0.1 * np.exp(-x[0])
    for i in range(1, dim):
        hessian[i, i] = 12 * np.power(x[i] - 1, 2)

    return hessian

def exponential_obj(dim):
    exp_val = lambda x: exponential(dim, x)
    exp_grad = lambda x: exponential_grad(dim, x)
    exp_hessian = lambda x: exponential_hessian(dim, x)
    return Obj(dim, exp_val, exp_grad, exp_hessian)

exponential_obj(3, np.array([1, 1, 1])).hessian(np.array([1, 1, 1]))
    



array([[0.00969746, 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ]])

In [78]:
def genhump(dim, x):
    # sum of sin(2 * x_i)^2 * sin(2 * x_{i+1})^2 + 0.05 * (x_i^2 + x_{i+1}^2)
    val = 0
    for i in range(dim - 1):
        val += np.power(np.sin(2 * x[i]), 2) * np.power(np.sin(2 * x[i+1]), 2) + 0.05 * (x[i]**2 + x[i+1]**2)

    return val

def genhump_grad(dim, x):   
    grad = np.zeros(dim)
    grad[0] = 4 * np.sin(2 * x[0]) * np.cos(2 * x[0]) * np.power(np.sin(2 * x[1]), 2) + 0.1 * x[0]
    for i in range(1, dim - 1): 
        grad[i] = 4 * np.sin(2 * x[i]) * np.cos(2 * x[i]) * np.power(np.sin(2 * x[i+1]), 2) + 0.1 * x[i]
        grad[i] += 4 * np.sin(2 * x[i]) * np.cos(2 * x[i]) * np.power(np.sin(2 * x[i-1]), 2) + 0.1 * x[i]

    grad[-1] = 4 * np.sin(2 * x[-1]) * np.cos(2 * x[-1]) * np.power(np.sin(2 * x[-2]), 2) + 0.1 * x[-1]
    return grad

def genhump_hess(dim, x):
    hess = np.zeros((dim, dim))
    hess[0, 0] = 8 * np.cos(2 * x[0]) * np.power(np.sin(2 * x[1]), 2) - 8 * np.power(np.sin(2 * x[0]), 2) * np.sin(2 * x[1]) * np.cos(2 * x[1]) + 0.1
    hess[0, 1] = 8 * np.sin(2 * x[0]) * np.cos(2 * x[0]) * np.sin(2 * x[1]) * np.cos(2 * x[1])
    for i in range(1, dim - 1):
        hess[i, i] = 8 * np.cos(2 * x[i]) * np.power(np.sin(2 * x[i+1]), 2) - 8 * np.power(np.sin(2 * x[i]), 2) * np.sin(2 * x[i+1]) * np.cos(2 * x[i+1]) + 0.1
        hess[i, i+1] = 8 * np.sin(2 * x[i]) * np.cos(2 * x[i]) * np.sin(2 * x[i+1]) * np.cos(2 * x[i+1])
        hess[i, i-1] = 8 * np.sin(2 * x[i]) * np.cos(2 * x[i]) * np.sin(2 * x[i-1]) * np.cos(2 * x[i-1])

    hess[-1, -1] = 8 * np.cos(2 * x[-1]) * np.power(np.sin(2 * x[-2]), 2) - 8 * np.power(np.sin(2 * x[-1]), 2) * np.sin(2 * x[-2]) * np.cos(2 * x[-2]) + 0.1
    hess[-1, -2] = 8 * np.sin(2 * x[-1]) * np.cos(2 * x[-1]) * np.sin(2 * x[-2]) * np.cos(2 * x[-2])
    return hess

def genhump_obj(dim):
    genhump_val = lambda x: genhump(dim, x)
    genhump_grad_val = lambda x: genhump_grad(dim, x)
    genhump_hess_val = lambda x: genhump_hess(dim, x)
    return Obj(dim, genhump_val, genhump_grad_val, genhump_hess_val)    

genhump_obj(3).hessian(np.array([1, 1, 1]))

array([[-0.14967101,  1.14550003,  0.        ],
       [ 1.14550003, -0.14967101,  1.14550003],
       [ 0.        ,  1.14550003, -0.14967101]])